In [ ]:
import os
import shutil

# Define the paths to the mask and image folders
mask_folder = 'Dataset_mask'
image_folder = 'Dataset_edge'

# Create a new folder to store the combined images and masks
combined_folder = 'Dataset_combined'
os.makedirs(combined_folder, exist_ok=True)

# Copy all files from the mask folder to the combined folder
for mask_file in os.listdir(mask_folder):
    shutil.copy2(os.path.join(mask_folder, mask_file), combined_folder)

# Copy all files from the image folder to the combined folder
for image_file in os.listdir(image_folder):
    shutil.copy2(os.path.join(image_folder, image_file), combined_folder)

print("Masks and images have been combined and stored in the 'Combined' folder.")

In [ ]:
import os
import glob
from PIL import Image
import os
import glob
from PIL import Image

class ParkingLotDataset(Dataset):
    def __init__(self, root_img, root_msk, pairs=None, transforms=None):
        self.root_img = root_img
        self.root_msk = root_msk
        self.transforms = transforms

        if pairs is None:
            # Get all image files
            self.image_paths = sorted(glob.glob(os.path.join(root_img, '*.png')))

            # Get all mask files
            self.mask_paths = sorted(glob.glob(os.path.join(root_msk, '*.png')))

            # Pair image and mask files based on their filenames
            #self.pairs = [(image_path, mask_path) for image_path in self.image_paths for mask_path in self.mask_paths if os.path.splitext(os.path.basename(image_path))[0] == os.path.splitext(os.path.basename(mask_path))[0]]
            self.pairs = []

            for image_path in self.image_paths:
                image_filename = os.path.splitext(os.path.basename(image_path))[0]
                for mask_path in self.mask_paths:
                    mask_filename = os.path.splitext(os.path.basename(mask_path))[0]
                    if image_filename in mask_filename:
                        self.pairs.append((image_path, mask_path))
                        break  # Se trova una corrispondenza, passa al prossimo file immagine
        else:
            self.pairs = pairs

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        image_path, mask_path = self.pairs[idx]

        # Load image
        image = Image.open(image_path)
        image_array = np.array(image)

        # Apply transformations
        if self.transforms:
            image_array = self.transforms(image_array)
            print(image_array.shape)
        

    


        return image_array, masks
        

        #merged_mask = np.zeros((2108, 2113), dtype=np.uint8)

        # Load and plot the masks
        #with open(mask_path, 'r') as f:
        #    masks_data = json.load(f)

        #for shape in masks_data['shapes']:
        #    points = np.array(shape['points'])
        #    mask = np.zeros((2108, 2113), dtype=np.uint8)
        #    cv2.fillPoly(mask, [points.astype(np.int32)], 1)
        #    _, binary_mask = cv2.threshold(mask, 0, 1, cv2.THRESH_BINARY)
        #    merged_mask += binary_mask

        #if self.transforms:
        #    masks = self.transforms(merged_mask)
        #    print(masks.shape)

        #print("Image shape:", image_array.shape)
        #print("Masks shape:", masks.shape)

plotting 1

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have a DataLoader object called train_loader
num_samples = 10

# Iterate over batches from the DataLoader
for batch_idx, (images, masks) in enumerate(train_loader):
    # Get the number of samples in the current batch
    batch_size = images.size(0)

    # Iterate over the samples in the batch
    for i in range(batch_size):
        # Convert the image tensor to a NumPy array
        image_array = images[i].permute(1, 2, 0).numpy()

        # Convert the mask tensor to a NumPy array
        mask_array = masks[i].squeeze().numpy()

        # Plot the image and the combined mask
        plt.figure(figsize=(12, 6))
        plt.subplot(1, 2, 1)
        plt.imshow(image_array, cmap='gray')
        plt.title(f'Image {batch_idx * batch_size + i + 1}')
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.imshow(image_array, cmap='gray')
        plt.imshow(mask_array, cmap='gray', alpha=0.5)
        plt.title(f'Image {batch_idx * batch_size + i + 1} with Masks')
        plt.axis('off')

        plt.show()

        # Break after the first 10 samples
        if batch_idx * batch_size + i == num_samples - 1:
            break

    # Break after processing one batch
    if batch_idx == 0:
        break

plotting 2

In [ ]:
import matplotlib.pyplot as plt

# Get the first pair of image and masks from the dataset
image_path, mask_path = train_dataset.pairs[0]
print(image_path)
print(mask_path)
print(train_dataset.pairs[0])

# Load the image
image = Image.open(image_path)
image_array = np.array(image)

# Plot the image
plt.figure(figsize=(12, 6))
plt.imshow(image_array, cmap='gray')
plt.title('Image')

merged_mask = np.zeros((2108, 2113), dtype=np.uint8)

# Load and plot the masks
with open(mask_path, 'r') as f:
    masks_data = json.load(f)

for i, shape in enumerate(masks_data['shapes']):
    points = np.array(shape['points'])
    mask = np.zeros((2108, 2113), dtype=np.uint8)
    cv2.fillPoly(mask, [points.astype(np.int32)], 1)
    _, binary_mask = cv2.threshold(mask, 0, 1, cv2.THRESH_BINARY)
    merged_mask += binary_mask
plt.imshow(merged_mask, cmap='gray', alpha=0.0)
plt.title(f'Image with Masks')
plt.show()


CALCOLATORE DI DIMENSIONI IMMAGINI

In [ ]:
#controllo il massimo valore di altezza e larghezza per fare il padding
import cv2
import os

# Directory containing the images
data_dir = 'Dataset_edge_smaller'

# Initialize variables to store max height and width
max_height = 0
max_width = 0

# Iterate through all images in the directory
for filename in os.listdir(data_dir):
    if filename.endswith('.png') or filename.endswith('.jpg'):
        # Load the image
        image_path = os.path.join(data_dir, filename)
        image = cv2.imread(image_path)

        # Get height and width of the image
        height, width, _ = image.shape
        if(height > 500 or width > 500):
            print(filename)
            print(image.shape)

        # Update max_height and max_width if necessary
        if height > max_height:
            max_height = height
        if width > max_width:
            max_width = width

        

#Image with different max_height and max_width: Queretaro-TEC_parcheggi.png
print("Maximum Height:", max_height)
print("Maximum Width:", max_width)

Image resize

In [ ]:
import cv2
import os

# Directory containing the images
data_dir = 'resized_masks512'
# Directory to save resized images
resized_dir = 'resized_masks512_bis'

# Ensure the resized_dir exists
os.makedirs(resized_dir, exist_ok=True)

# Iterate through all images in the directory
for filename in os.listdir(data_dir):
    if filename.endswith('.png') or filename.endswith('.jpg'):
        # Load the image
        image_path = os.path.join(data_dir, filename)
        image = cv2.imread(image_path)

        # Get height and width of the image
        height, width, _ = image.shape

        # If the image is larger than 1000 in either dimension, resize it
        if height > 512 or width > 512:
            # Calculate the scale factor
            scale_factor = 0.5  # Reduce size by half

            # Calculate the new dimensions of the image
            new_width = int(width * scale_factor)
            new_height = int(height * scale_factor)

            # Resize the image
            image = cv2.resize(image, (new_width, new_height))

        # Save the image (resized or original) to the new directory
        new_image_path = os.path.join(resized_dir, filename)
        cv2.imwrite(new_image_path, image)
        

Mask resize and padding

In [ ]:
import cv2
import os

# Directory containing the images
data_dir = 'Resized_masks512'
# Directory to save resized images
resized_dir = 'Resizedmasks_smaller'

# Ensure the resized_dir exists
os.makedirs(resized_dir, exist_ok=True)

# Iterate through all images in the directory
for filename in os.listdir(data_dir):
    if filename.endswith('.png') or filename.endswith('.jpg'):
        # Load the image
        image_path = os.path.join(data_dir, filename)
        image = cv2.imread(image_path)

        # Get height and width of the image
        height, width, _ = image.shape

        # If the image is larger than 1000 in either dimension, resize it
        if height > 512 or width > 512:
            # Calculate the scale factor
            scale_factor = 0.5  # Reduce size by half

            # Calculate the new dimensions of the image
            new_width = int(width * scale_factor)
            new_height = int(height * scale_factor)

            # Resize the image
            image = cv2.resize(image, (new_width, new_height))

        ## MASK PADDING
        max_height = 512
        max_width = 512
        padding_height = max(0, max_height - image.shape[0])
        padding_width = max(0, max_width - image.shape[1])
        mask_padded = cv2.copyMakeBorder(image, 0, padding_height, 0, padding_width, cv2.BORDER_CONSTANT, value=0)
        # Save the image (resized or original) to the new directory
        new_image_path = os.path.join(resized_dir, filename)
        cv2.imwrite(new_image_path, mask_padded)


Mask resize and not pad

In [5]:
import os
from PIL import Image
import cv2
import numpy as np

# Directory contenente le immagini da ridimensionare
input_dir = "image_newmask"

# Directory di output dove verranno salvate le immagini ridimensionate
output_dir = "Resizedmasks_nopad"

# Dimensioni desiderate delle immagini ridimensionate
target_size = (512, 512)

# Assicurati che la directory di output esista, altrimenti creala
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Lista dei formati di immagine supportati da Pillow
supported_formats = [".jpg", ".jpeg", ".png", ".bmp", ".gif"]

# Loop attraverso tutti i file nella directory di input
for filename in os.listdir(input_dir):
    # Verifica se il file è un'immagine supportata
    if any(filename.lower().endswith(ext) for ext in supported_formats):
        # Percorso completo del file di input
        input_path = os.path.join(input_dir, filename)
        
        # Carica l'immagine utilizzando Pillow
        image = Image.open(input_path)
        
        # Ridimensiona l'immagine alla dimensione desiderata
        resized_image = image.resize(target_size)
        
        # Converti l'immagine ridimensionata in un array numpy
        resized_array = np.array(resized_image)

        output_image_path = os.path.join(output_dir, filename)
        cv2.imwrite(output_image_path, resized_array)
            
print("Ridimensionamento completato.")


Ridimensionamento completato.


Dimension checker

In [33]:
for filename in os.listdir('RGB_dataset'):
    image_path = os.path.join('RGB_dataset', filename)
    image = cv2.imread(image_path)
    if image.shape[0] > 510 or image.shape[1] > 510:
        print(image_path)
        print(image.shape[0],image.shape[1])

RGB_dataset\101881731.png
512 512
RGB_dataset\103109948.png
512 512
RGB_dataset\103614205.png
512 512
RGB_dataset\104101927.png
512 512
RGB_dataset\104357814.png
512 512
RGB_dataset\104357815.png
512 512
RGB_dataset\104357817.png
512 512
RGB_dataset\104907575.png
512 512
RGB_dataset\116016718.png
512 512
RGB_dataset\119422952.png
512 512
RGB_dataset\119917133.png
512 512
RGB_dataset\126381586.png
512 512
RGB_dataset\126381588.png
512 512
RGB_dataset\126381594.png
512 512
RGB_dataset\127649594.png
512 512
RGB_dataset\127649606.png
512 512
RGB_dataset\130730813.png
512 512
RGB_dataset\132613254.png
512 512
RGB_dataset\134714700.png
512 512
RGB_dataset\136128142.png
512 512
RGB_dataset\136128143.png
512 512
RGB_dataset\136128144.png
512 512
RGB_dataset\137007454.png
512 512
RGB_dataset\137007455.png
512 512
RGB_dataset\139146552.png
512 512
RGB_dataset\139146568.png
512 512
RGB_dataset\144384815.png
512 512
RGB_dataset\144848258.png
512 512
RGB_dataset\147397541.png
512 512
RGB_dataset\14

512x512 no padding

In [13]:
import os
from PIL import Image
import cv2
import numpy as np

# Directory contenente le immagini da ridimensionare
input_dir = "dataset"

# Directory di output dove verranno salvate le immagini ridimensionate
output_dir = "Resize_no_padding"

# Dimensioni desiderate delle immagini ridimensionate
target_size = (512, 512)

# Assicurati che la directory di output esista, altrimenti creala
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Lista dei formati di immagine supportati da Pillow
supported_formats = [".jpg", ".jpeg", ".png", ".bmp", ".gif"]

# Loop attraverso tutti i file nella directory di input
for filename in os.listdir(input_dir):
    # Verifica se il file è un'immagine supportata
    if any(filename.lower().endswith(ext) for ext in supported_formats):
        # Percorso completo del file di input
        input_path = os.path.join(input_dir, filename)
        
        # Carica l'immagine utilizzando Pillow
        image = Image.open(input_path)
        
        # Ridimensiona l'immagine alla dimensione desiderata
        resized_image = image.resize(target_size)
        
        # Converti l'immagine ridimensionata in un array numpy
        resized_array = np.array(resized_image)
        
        if "_testing" in filename or "_training" in filename:
            # Rinominare il file rimuovendo "_testing" o "_training" dal nome
            new_filename = filename.replace("_testing", "").replace("_training", "")
            
            print(f"Rinominato il file {filename} in {new_filename}")
            # Salva l'immagine del bordo nella cartella di output
            output_image_path = os.path.join(output_dir, new_filename)
            cv2.imwrite(output_image_path, resized_array)
            
print("Ridimensionamento completato.")


Rinominato il file 101881731_training.png in 101881731.png
Rinominato il file 103109948_testing.png in 103109948.png
Rinominato il file 103614205_training.png in 103614205.png
Rinominato il file 104101927_testing.png in 104101927.png
Rinominato il file 104357814_training.png in 104357814.png
Rinominato il file 104357815_training.png in 104357815.png
Rinominato il file 104357817_training.png in 104357817.png
Rinominato il file 104907575_testing.png in 104907575.png
Rinominato il file 116016718_training.png in 116016718.png
Rinominato il file 119422952_training.png in 119422952.png
Rinominato il file 119917133_training.png in 119917133.png
Rinominato il file 126381586_training.png in 126381586.png
Rinominato il file 126381588_training.png in 126381588.png
Rinominato il file 126381594_training.png in 126381594.png
Rinominato il file 127649594_training.png in 127649594.png
Rinominato il file 127649606_testing.png in 127649606.png
Rinominato il file 130730813_training.png in 130730813.png
R

Store canny edge

In [ ]:
dataset_path = 'ResizedDataset_even_smaller'
edge_path = 'Dataset_edge_smaller'

os.makedirs(edge_path, exist_ok=True)

def get_edge_image(image, low_threshold, high_threshold):
    # Convert the image to grayscale if it's not already
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Apply Canny edge detection
    edges = cv2.Canny(gray, low_threshold, high_threshold)
    
    return edges
#edge detection + padding immagini
for filename in os.listdir(dataset_path):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        
        input_image_path = os.path.join(dataset_path, filename)
        image = cv2.imread(input_image_path)
        # Ottieni l'immagine del bordo
        edge_image = get_edge_image(image, low_threshold=200, high_threshold=300)
        
        # Apply padding to the edge image
        max_height = 512
        max_width = 512
        padding_height = max(0, max_height - edge_image.shape[0])
        padding_width = max(0, max_width - edge_image.shape[1])
        edge_image_padded = cv2.copyMakeBorder(edge_image, 0, padding_height, 0, padding_width, cv2.BORDER_CONSTANT, value=0)
        #Verificare se il nome del file contiene "_testing" o "_training"
        if "_testing" in filename or "_training" in filename:
            # Rinominare il file rimuovendo "_testing" o "_training" dal nome
            new_filename = filename.replace("_testing", "").replace("_training", "")
            
            print(f"Rinominato il file {filename} in {new_filename}")
            # Salva l'immagine del bordo nella cartella di output
            output_image_path = os.path.join(edge_path, new_filename)
            cv2.imwrite(output_image_path, edge_image_padded)

print("Il rilevamento dei bordi è stato applicato a tutte le immagini nella cartella.")


RGB

In [32]:
dataset_path = 'ResizedDataset_even_smaller'
edge_path = 'RGB_dataset'

os.makedirs(edge_path, exist_ok=True)

for filename in os.listdir(dataset_path):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        
        input_image_path = os.path.join(dataset_path, filename)
        image = cv2.imread(input_image_path)

        # Apply padding to the edge image
        max_height = 512
        max_width = 512
        padding_height = max(0, max_height - image.shape[0])
        padding_width = max(0, max_width - image.shape[1])
        image_padded = cv2.copyMakeBorder(image, 0, padding_height, 0, padding_width, cv2.BORDER_CONSTANT, value=0)
        #Verificare se il nome del file contiene "_testing" o "_training"
        if "_testing" in filename or "_training" in filename:
            # Rinominare il file rimuovendo "_testing" o "_training" dal nome
            new_filename = filename.replace("_testing", "").replace("_training", "")
            
            print(f"Rinominato il file {filename} in {new_filename}")
            # Salva l'immagine del bordo nella cartella di output
            output_image_path = os.path.join(edge_path, new_filename)
            cv2.imwrite(output_image_path, image_padded)

Rinominato il file 101881731_training.png in 101881731.png
Rinominato il file 103109948_testing.png in 103109948.png
Rinominato il file 103614205_training.png in 103614205.png
Rinominato il file 104101927_testing.png in 104101927.png
Rinominato il file 104357814_training.png in 104357814.png
Rinominato il file 104357815_training.png in 104357815.png
Rinominato il file 104357817_training.png in 104357817.png
Rinominato il file 104907575_testing.png in 104907575.png
Rinominato il file 116016718_training.png in 116016718.png
Rinominato il file 119422952_training.png in 119422952.png
Rinominato il file 119917133_training.png in 119917133.png
Rinominato il file 126381586_training.png in 126381586.png
Rinominato il file 126381588_training.png in 126381588.png
Rinominato il file 126381594_training.png in 126381594.png
Rinominato il file 127649594_training.png in 127649594.png
Rinominato il file 127649606_testing.png in 127649606.png
Rinominato il file 130730813_training.png in 130730813.png
R

RGB normalizzazione

In [38]:
from PIL import Image
import os
import numpy as np

# Definisci il percorso della cartella contenente le immagini
folder_path = 'RGB_dataset'

# Inizializza le liste per i canali RGB
red_values = []
green_values = []
blue_values = []

# Itera attraverso le immagini nella cartella
for filename in os.listdir(folder_path):
    # Verifica se il file è un'immagine
    if filename.endswith(".jpg") or filename.endswith(".png"):
        # Carica l'immagine
        image = Image.open(os.path.join(folder_path, filename))
        # Estrai i valori dei pixel
        pixels = np.array(image)
        # Calcola la media dei valori dei pixel per ciascun canale di colore
        red_values.append(np.mean(pixels[:,:,0]))
        green_values.append(np.mean(pixels[:,:,1]))
        blue_values.append(np.mean(pixels[:,:,2]))

# Calcola la media e la deviazione standard per ogni canale di colore
mean_red = np.mean(red_values)
mean_green = np.mean(green_values)
mean_blue = np.mean(blue_values)

std_red = np.std(red_values)
std_green = np.std(green_values)
std_blue = np.std(blue_values)

print("Mean Red:", mean_red)
print("Mean Green:", mean_green)
print("Mean Blue:", mean_blue)

print("Std Red:", std_red)
print("Std Green:", std_green)
print("Std Blue:", std_blue)


Mean Red: 35.54499132537842
Mean Green: 35.16268335723877
Mean Blue: 33.40791042327881
Std Red: 21.848256784304716
Std Green: 21.59419671163517
Std Blue: 20.93842322158817


RGB normalizzazione 2

In [3]:
import numpy as np
from PIL import Image
import os
import glob

def calculate_mean_std(image_paths):
    pixel_values = []
    for image_path in image_paths:
        image = Image.open(image_path).convert('RGB')
        image_array = np.array(image)
        pixel_values.extend(image_array.reshape(-1, 3))

    pixel_values = np.array(pixel_values, dtype=np.float32)
    mean = pixel_values.mean(axis=0) / 255.0
    std = pixel_values.std(axis=0) / 255.0
    return mean, std

# Get all image paths
image_paths = sorted(glob.glob(os.path.join('test_images', '*.png')))

# Calculate mean and std
mean, std = calculate_mean_std(image_paths)
print(f"Mean: {mean}")
print(f"Std: {std}")

Mean: [0.39239654 0.40764114 0.41015136]
Std: [0.20021597 0.20296764 0.2084963 ]


In [5]:
import os
from PIL import Image
import numpy as np

def calculate_mean_std(image_dir):
    image_paths = []
    for filename in os.listdir(image_dir):
        image_path = os.path.join(image_dir, filename)
        if os.path.isfile(image_path):
            image_paths.append(image_path)

    pixel_values = []
    for image_path in image_paths:
        image = Image.open(image_path).convert('RGB')
        image_array = np.array(image, dtype=np.uint8)  # Ensure pixel values are integers
        pixel_values.extend(image_array.reshape(-1, 3))
    pixel_values = np.array(pixel_values, dtype=np.uint8)  # Ensure pixel values are integers
    mean = pixel_values.mean(axis=0)
    std = pixel_values.std(axis=0)
    return mean, std

# Assuming your images are in a directory named 'Dataset' in the current working directory
image_dir = 'train_images'
mean, std = calculate_mean_std(image_dir)
print(f"Mean: {mean}")
print(f"Std: {std}")

Mean: [35.54499133 35.16268336 33.40791042]
Std: [57.87239911 56.89318723 54.54667448]


DATAAUGGGGG

In [19]:
import os
from PIL import Image
from tqdm import tqdm
import torchvision.transforms as transforms
import torchvision.transforms.functional as F
import torch
import os
import shutil
from PIL import Image
from tqdm import tqdm
import torchvision.transforms as transforms
import torchvision.transforms.functional as F
import torch
import random


# Define synchronized transformation class
class SynchronizedTransform:
    def __init__(self):
        self.color_transforms = transforms.ColorJitter(
            brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1
        )
        self.to_tensor = transforms.ToTensor()
        self.resize = transforms.Resize(size=(512, 512))
        self.to_pil = transforms.ToPILImage()

    def __call__(self, image, mask):
        # Convert to tensor
        image = self.to_tensor(image)
        mask = self.to_tensor(mask)

        # Apply Resize
        image = self.resize(image)
        mask = self.resize(mask)

        # Apply Random Horizontal Flip
        if torch.rand(1) < 0.5:
            image = F.hflip(image)
            mask = F.hflip(mask)

        # Apply Random Vertical Flip
        if torch.rand(1) < 0.5:
            image = F.vflip(image)
            mask = F.vflip(mask)

        # Apply Random Rotation
        angle = transforms.RandomRotation.get_params([-10, 10])
        image = F.rotate(image, angle)
        mask = F.rotate(mask, angle, interpolation=F.InterpolationMode.NEAREST)

        # Apply Random Resized Crop
        i, j, h, w = transforms.RandomResizedCrop.get_params(image, scale=(0.8, 1.0), ratio=(3.0/4.0, 4.0/3.0))
        image = F.resized_crop(image, i, j, h, w, size=(512, 512))
        mask = F.resized_crop(mask, i, j, h, w, size=(512, 512), interpolation=F.InterpolationMode.NEAREST)

        # Apply color transforms only to the image
        image = self.color_transforms(image)

        # Convert back to PIL
        image = self.to_pil(image)
        mask = self.to_pil(mask)

        return image, mask

sync_transform = SynchronizedTransform()

def save_image(image, path, filename):
    image.save(os.path.join(path, filename))

def augment_and_save(image_path, mask_path, save_image_path, save_mask_path, num_augmentations=3):
    #image_files = sorted(os.listdir(image_path))
    #mask_files = sorted(os.listdir(mask_path))

    if not os.path.exists(save_image_path):
        os.makedirs(save_image_path)
    if not os.path.exists(save_mask_path):
        os.makedirs(save_mask_path)

    current_index = 0

    
    img = Image.open(image_path).convert("RGB")
    mask = Image.open(mask_path).convert("L")

    # Save original image and mask
    save_image(img, save_image_path, f"{current_index}.png")
    save_image(mask, save_mask_path, f"{current_index}_SegmentationClass.png")
    #current_index += 1

    for _ in range(num_augmentations):
        augmented_img, augmented_mask = sync_transform(img, mask)

        save_image(augmented_img, save_image_path, f"{current_index}.png")
        save_image(augmented_mask, save_mask_path, f"{current_index}_SegmentationClass.png")
        current_index += 1

# Percorsi delle cartelle
#image_path = 'Resize_no_padding'
image_path1 = 'C:/Users/loren/Documents/GitHub/Parking-lot/Resize_no_padding_1'
image_path2 = 'C:/Users/loren/Documents/GitHub/Parking-lot/Resize_no_padding_2',
image_path3 = 'C:/Users/loren/Documents/GitHub/Parking-lot/Resize_no_padding_3'
mask_path = 'Resizedmasks_nopad'

# Crea le cartelle di destinazione
train_image_path = 'train_images'
train_mask_path = 'train_masks'
val_image_path = 'val_images'
val_mask_path = 'val_masks'
test_image_path = 'test_images'
test_mask_path = 'test_masks'

os.makedirs(train_image_path, exist_ok=True)
os.makedirs(train_mask_path, exist_ok=True)
os.makedirs(val_image_path, exist_ok=True)
os.makedirs(val_mask_path, exist_ok=True)
os.makedirs(test_image_path, exist_ok=True)
os.makedirs(test_mask_path, exist_ok=True)

transform = transforms.Compose([
    transforms.ToTensor(),
])

mask_transforms = transforms.Compose([
    transforms.ToTensor(),
])

# Crea il dataset
dataset1 = ParkingLotDataset(image_path1, mask_path, transforms=transform, mask_transforms=mask_transforms)
dataset2 = ParkingLotDataset(image_path2, mask_path, transforms=transform, mask_transforms=mask_transforms)
dataset3 = ParkingLotDataset(image_path3, mask_path, transforms=transform, mask_transforms=mask_transforms)

# Definisci le proporzioni per la divisione
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1
total_size = len(dataset1)+len(dataset2)+len(dataset3)
print(total_size)

# Mischia il dataset
random.shuffle(dataset1.pairs)
random.shuffle(dataset2.pairs)
random.shuffle(dataset3.pairs)

# Calcola le dimensioni di ogni split
train_size = int(train_ratio * total_size)
val_size = int(val_ratio * total_size)
test_size = total_size - train_size - val_size

# Dividi il dataset
train_pairs = dataset1.pairs[:len(dataset1)*0.7]+dataset2.pairs[:len(dataset2)*0.7]+dataset3.pairs[:len(dataset3)*0.7]
val_pairs = dataset.pairs[train_size:train_size+val_size]
test_pairs = dataset.pairs[train_size+val_size:]

# Applica data augmentation e salva il train dataset
# Applica data augmentation e salva il train dataset
for i, (image_path, mask_path) in enumerate(tqdm(train_pairs, desc="Augmenting train dataset")):
    augment_and_save(image_path, mask_path, os.path.join(train_image_path, str(i)), os.path.join(train_mask_path, str(i)))

# Salva il val dataset
for i, (image_path, mask_path) in enumerate(tqdm(val_pairs, desc="Saving val dataset")):
    image = Image.open(image_path)
    mask = Image.open(mask_path)
    save_image(image, val_image_path, f"{i}.png")
    save_image(mask, val_mask_path, f"{i}_SegmentationClass.png")

# Salva il test dataset
for i, (image_path, mask_path) in enumerate(tqdm(test_pairs, desc="Saving test dataset")):
    image = Image.open(image_path)
    mask = Image.open(mask_path)
    save_image(image, test_image_path, f"{i}.png")
    save_image(mask, test_mask_path, f"{i}_SegmentationClass.png")


NameError: name 'ParkingLotDataset' is not defined

image train manipulation

In [24]:
import os
import shutil

def flatten_directory(directory):
    # Counter for renaming files
    file_counter = 1

    # Walk through the directory tree
    for root, dirs, files in os.walk(directory, topdown=False):
        for file in files:
            # Generate the new file name
            new_file_name = f"{file_counter:04d}{os.path.splitext(file)[1]}"
            new_file_path = os.path.join(directory, new_file_name)
            
            # Move the file to the root directory and rename it
            old_file_path = os.path.join(root, file)
            shutil.move(old_file_path, new_file_path)
            
            print(f"Moved: {old_file_path} -> {new_file_path}")
            file_counter += 1
        
        # Remove empty directories
        for dir in dirs:
            dir_path = os.path.join(root, dir)
            try:
                os.rmdir(dir_path)
                print(f"Removed empty directory: {dir_path}")
            except OSError as e:
                print(f"Error removing directory {dir_path}: {e}")

# Path to the directory you want to flatten
directory_path = 'train_images_2'
flatten_directory(directory_path)


Moved: train_images_2\0\0.png -> train_images_2\0001.png
Moved: train_images_2\0\1.png -> train_images_2\0002.png
Moved: train_images_2\0\2.png -> train_images_2\0003.png
Moved: train_images_2\1\0.png -> train_images_2\0004.png
Moved: train_images_2\1\1.png -> train_images_2\0005.png
Moved: train_images_2\1\2.png -> train_images_2\0006.png
Moved: train_images_2\10\0.png -> train_images_2\0007.png
Moved: train_images_2\10\1.png -> train_images_2\0008.png
Moved: train_images_2\10\2.png -> train_images_2\0009.png
Moved: train_images_2\100\0.png -> train_images_2\0010.png
Moved: train_images_2\100\1.png -> train_images_2\0011.png
Moved: train_images_2\100\2.png -> train_images_2\0012.png
Moved: train_images_2\101\0.png -> train_images_2\0013.png
Moved: train_images_2\101\1.png -> train_images_2\0014.png
Moved: train_images_2\101\2.png -> train_images_2\0015.png
Moved: train_images_2\102\0.png -> train_images_2\0016.png
Moved: train_images_2\102\1.png -> train_images_2\0017.png
Moved: train

mask train manipulation

In [25]:
import os
import shutil

def flatten_directory(directory):
    # Counter for renaming files
    file_counter = 1

    # Walk through the directory tree
    for root, dirs, files in os.walk(directory, topdown=False):
        for file in files:
            # Generate the new file name with the suffix
            file_name, file_extension = os.path.splitext(file)
            new_file_name = f"{file_counter:04d}_SegmentationClass{file_extension}"
            new_file_path = os.path.join(directory, new_file_name)
            
            # Move the file to the root directory and rename it
            old_file_path = os.path.join(root, file)
            shutil.move(old_file_path, new_file_path)
            
            print(f"Moved: {old_file_path} -> {new_file_path}")
            file_counter += 1
        
        # Remove empty directories
        for dir in dirs:
            dir_path = os.path.join(root, dir)
            try:
                os.rmdir(dir_path)
                print(f"Removed empty directory: {dir_path}")
            except OSError as e:
                print(f"Error removing directory {dir_path}: {e}")

# Path to the directory you want to flatten
directory_path = 'train_masks'
flatten_directory(directory_path)


Moved: train_masks\0\0_SegmentationClass.png -> train_masks\0001_SegmentationClass.png
Moved: train_masks\0\1_SegmentationClass.png -> train_masks\0002_SegmentationClass.png
Moved: train_masks\0\2_SegmentationClass.png -> train_masks\0003_SegmentationClass.png
Moved: train_masks\1\0_SegmentationClass.png -> train_masks\0004_SegmentationClass.png
Moved: train_masks\1\1_SegmentationClass.png -> train_masks\0005_SegmentationClass.png
Moved: train_masks\1\2_SegmentationClass.png -> train_masks\0006_SegmentationClass.png
Moved: train_masks\10\0_SegmentationClass.png -> train_masks\0007_SegmentationClass.png
Moved: train_masks\10\1_SegmentationClass.png -> train_masks\0008_SegmentationClass.png
Moved: train_masks\10\2_SegmentationClass.png -> train_masks\0009_SegmentationClass.png
Moved: train_masks\100\0_SegmentationClass.png -> train_masks\0010_SegmentationClass.png
Moved: train_masks\100\1_SegmentationClass.png -> train_masks\0011_SegmentationClass.png
Moved: train_masks\100\2_Segmentatio

Gradazione

In [5]:
import os
import re

# Percorsi
cartella_immagini = r"C:\Users\loren\Documents\GitHub\Parking-lot\Resize_no_padding"
file_valori = r"C:\Users\loren\Desktop\livelli_park.txt"

# Leggi i valori dal file txt
with open(file_valori, 'r') as f:
    valori = f.read().splitlines()

# Ottieni la lista dei file nella cartella immagini
file_immagini = os.listdir(cartella_immagini)

# Funzione per estrarre il numero dal nome del file
def estrai_numero(nome_file):
    match = re.search(r'(\d+)', nome_file)
    return int(match.group(1)) if match else float('inf')

# Ordina i file basandosi sul numero estratto
file_immagini.sort(key=estrai_numero)

# Controlla che ci siano tanti valori quanti sono i file
if len(valori) != len(file_immagini):
    raise ValueError("Il numero di valori nel file txt non corrisponde al numero di immagini.")

# Funzione per rimuovere eventuali suffissi "_1", "_2" o "_3"
def rimuovi_suffisso(nome_file):
    return re.sub(r'_[1-3]$', '', nome_file)

# Rinominazione dei file
for i, filename in enumerate(file_immagini):
    valore = valori[i]
    # Estrai il nome e l'estensione del file
    nome, estensione = os.path.splitext(filename)
    # Rimuovi eventuali suffissi esistenti
    nome_pulito = rimuovi_suffisso(nome)
    # Nuovo nome del file
    nuovo_nome = f"{nome_pulito}_{valore}{estensione}"
    # Percorsi completi per il rinominamento
    vecchio_percorso = os.path.join(cartella_immagini, filename)
    nuovo_percorso = os.path.join(cartella_immagini, nuovo_nome)
    # Rinomina il file
    os.rename(vecchio_percorso, nuovo_percorso)

print("Rinomina completata!")


Rinomina completata!


In [6]:
import os
import shutil
import re

# Percorsi
cartella_immagini = r"C:\Users\loren\Documents\GitHub\Parking-lot\Resize_no_padding"
cartella_destinazione_1 = r"C:\Users\loren\Documents\GitHub\Parking-lot\Resize_no_padding_1"
cartella_destinazione_2 = r"C:\Users\loren\Documents\GitHub\Parking-lot\Resize_no_padding_2"
cartella_destinazione_3 = r"C:\Users\loren\Documents\GitHub\Parking-lot\Resize_no_padding_3"

# Creare le cartelle di destinazione se non esistono
os.makedirs(cartella_destinazione_1, exist_ok=True)
os.makedirs(cartella_destinazione_2, exist_ok=True)
os.makedirs(cartella_destinazione_3, exist_ok=True)

# Ottieni la lista dei file nella cartella immagini
file_immagini = os.listdir(cartella_immagini)

# Funzione per identificare il suffisso
def identifica_suffisso(nome_file):
    match = re.search(r'_([1-3])$', nome_file)
    return match.group(1) if match else None

# Spostamento dei file nelle rispettive cartelle
for filename in file_immagini:
    # Estrai il suffisso
    suffisso = identifica_suffisso(os.path.splitext(filename)[0])
    
    if suffisso:
        # Determina la cartella di destinazione in base al suffisso
        if suffisso == '1':
            destinazione = cartella_destinazione_1
        elif suffisso == '2':
            destinazione = cartella_destinazione_2
        elif suffisso == '3':
            destinazione = cartella_destinazione_3
        
        # Percorsi completi per lo spostamento
        vecchio_percorso = os.path.join(cartella_immagini, filename)
        nuovo_percorso = os.path.join(destinazione, filename)
        
        # Sposta il file
        shutil.move(vecchio_percorso, nuovo_percorso)

print("Spostamento completato!")


Spostamento completato!


Data_augmentation_immagini_gradate

In [15]:
import os
from PIL import Image
from tqdm import tqdm
import torchvision.transforms as transforms
import torchvision.transforms.functional as F
import torch
import random

# Define synchronized transformation class
class SynchronizedTransform:
    def __init__(self):
        self.color_transforms = transforms.ColorJitter(
            brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1
        )
        self.to_tensor = transforms.ToTensor()
        self.resize = transforms.Resize(size=(512, 512))
        self.to_pil = transforms.ToPILImage()

    def __call__(self, image, mask):
        # Convert to tensor
        image = self.to_tensor(image)
        mask = self.to_tensor(mask)

        # Apply Resize
        image = self.resize(image)
        mask = self.resize(mask)

        # Apply Random Horizontal Flip
        if torch.rand(1) < 0.5:
            image = F.hflip(image)
            mask = F.hflip(mask)

        # Apply Random Vertical Flip
        if torch.rand(1) < 0.5:
            image = F.vflip(image)
            mask = F.vflip(mask)

        # Apply Random Rotation
        angle = transforms.RandomRotation.get_params([-10, 10])
        image = F.rotate(image, angle)
        mask = F.rotate(mask, angle, interpolation=F.InterpolationMode.NEAREST)

        # Apply Random Resized Crop
        i, j, h, w = transforms.RandomResizedCrop.get_params(image, scale=(0.8, 1.0), ratio=(3.0/4.0, 4.0/3.0))
        image = F.resized_crop(image, i, j, h, w, size=(512, 512))
        mask = F.resized_crop(mask, i, j, h, w, size=(512, 512), interpolation=F.InterpolationMode.NEAREST)

        # Apply color transforms only to the image
        image = self.color_transforms(image)

        # Convert back to PIL
        image = self.to_pil(image)
        mask = self.to_pil(mask)

        return image, mask

sync_transform = SynchronizedTransform()

def save_image(image, path, filename):
    image.save(os.path.join(path, filename))

def augment_and_save(image_path, mask_path, save_image_path, save_mask_path, num_augmentations=3):
    if not os.path.exists(save_image_path):
        os.makedirs(save_image_path)
    if not os.path.exists(save_mask_path):
        os.makedirs(save_mask_path)

    img = Image.open(image_path).convert("RGB")
    mask = Image.open(mask_path).convert("L")

    current_index = 0

    # Save original image and mask
    save_image(img, save_image_path, f"{current_index}.png")
    save_image(mask, save_mask_path, f"{current_index}_SegmentationClass.png")
    current_index += 1

    for _ in range(num_augmentations):
        for i in range(3):  # Save 3 augmented versions
            augmented_img, augmented_mask = sync_transform(img, mask)
            save_image(augmented_img, save_image_path, f"{current_index}.png")
            save_image(augmented_mask, save_mask_path, f"{current_index}_SegmentationClass.png")
            current_index += 1

# Percorsi delle cartelle delle immagini e delle maschere
image_folders = [
    'C:/Users/loren/Documents/GitHub/Parking-lot/Resize_no_padding_1',
    'C:/Users/loren/Documents/GitHub/Parking-lot/Resize_no_padding_2',
    'C:/Users/loren/Documents/GitHub/Parking-lot/Resize_no_padding_3'
]
mask_folder = 'Resizedmasks_nopad'

# Crea le cartelle per i set di dati
train_image_path = 'train_images'
train_mask_path = 'train_masks'
val_image_path = 'val_images'
val_mask_path = 'val_masks'
test_image_path = 'test_images'
test_mask_path = 'test_masks'

os.makedirs(train_image_path, exist_ok=True)
os.makedirs(train_mask_path, exist_ok=True)
os.makedirs(val_image_path, exist_ok=True)
os.makedirs(val_mask_path, exist_ok=True)
os.makedirs(test_image_path, exist_ok=True)
os.makedirs(test_mask_path, exist_ok=True)

# Funzione per dividere i file in train, validation e test set
def split_data(image_folders, mask_folder):
    train_pairs = []
    val_pairs = []
    test_pairs = []

    total_images = 0
    for folder in image_folders:
        image_files = [f for f in os.listdir(folder) if f.endswith('.png')]
        total_images += len(image_files)

    train_size = 1050  # Triplicato per le data augmentations

    for folder in image_folders:
        image_files = sorted([f for f in os.listdir(folder) if f.endswith('.png')])
        mask_files = sorted([f for f in os.listdir(mask_folder) if f.endswith('.png')])

        image_files_base = [os.path.splitext(f)[0] for f in image_files]
        mask_files_base = [os.path.splitext(f)[0].replace('_SegmentationClass', '') for f in mask_files]

        # Verifica che ogni immagine abbia una maschera corrispondente
        paired_files = [(img, mask) for img in image_files_base for mask in mask_files_base if img.split('_')[0] == mask]

        assert len(paired_files) == len(image_files), f"Mismatch in {folder}: {len(image_files)} images and {len(mask_files)} masks."

        # Mischia i file
        random.shuffle(paired_files)

        # Calcola le dimensioni di ogni split
        folder_size = len(paired_files)
        folder_train_size = int(folder_size * train_size / total_images)
        folder_val_size = int(0.2 * folder_size)
        folder_test_size = folder_size - folder_train_size - folder_val_size

        train_files = paired_files[:folder_train_size]
        val_files = paired_files[folder_train_size:folder_train_size + folder_val_size]
        test_files = paired_files[folder_train_size + folder_val_size:]

        train_pairs.extend([(os.path.join(folder, f"{img}.png"), os.path.join(mask_folder, f"{mask}_SegmentationClass.png")) for img, mask in train_files])
        val_pairs.extend([(os.path.join(folder, f"{img}.png"), os.path.join(mask_folder, f"{mask}_SegmentationClass.png")) for img, mask in val_files])
        test_pairs.extend([(os.path.join(folder, f"{img}.png"), os.path.join(mask_folder, f"{mask}_SegmentationClass.png")) for img, mask in test_files])

    return train_pairs, val_pairs, test_pairs

# Definisci le proporzioni per la divisione
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

# Dividi le immagini delle cartelle tra train, val e test
train_pairs, val_pairs, test_pairs = split_data(image_folders, mask_folder)

# Applica data augmentation e salva il train dataset
for i, (image_path, mask_path) in enumerate(tqdm(train_pairs, desc="Augmenting and saving train dataset")):
    augment_and_save(image_path, mask_path, train_image_path, train_mask_path, num_augmentations=3)

# Salva il val dataset
for i, (image_path, mask_path) in enumerate(tqdm(val_pairs, desc="Saving val dataset")):
    image = Image.open(image_path)
    mask = Image.open(mask_path)
    save_image(image, val_image_path, f"{i}.png")
    save_image(mask, val_mask_path, f"{i}_SegmentationClass.png")

# Salva il test dataset
for i, (image_path, mask_path) in enumerate(tqdm(test_pairs, desc="Saving test dataset")):
    image = Image.open(image_path)
    mask = Image.open(mask_path)
    save_image(image, test_image_path, f"{i}.png")
    save_image(mask, test_mask_path, f"{i}_SegmentationClass.png")

Augmenting and saving train dataset:   0%|          | 2/499 [00:05<23:42,  2.86s/it]


KeyboardInterrupt: 